In [1]:
# Import necessary libraries
import os
import shutil


# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import glob
import matplotlib.pyplot as plt
import json
import csv


import numpy as np
import seaborn as sns


Mounted at /content/drive


In [2]:
import re

folder_path = '/content/drive/MyDrive/ADSDB/landing/temporal'

INFLATION_FILENAME = "inflation_rate"
EMPLOYMENT_FILENAME = "employment_rate"
HPI_FILENAME = "house_price_index"
HPI_WEIGHTS_FILENAME = "house_price_index_weights"

FILENAMES = [INFLATION_FILENAME, EMPLOYMENT_FILENAME, HPI_FILENAME, HPI_WEIGHTS_FILENAME]


In [106]:
df_inflation = pd.read_csv('/content/drive/MyDrive/ADSDB/landing/temporal/inflation_rate.csv', sep=';')


In [107]:
# Strip leading spaces from column names
df_inflation.columns = df_inflation.columns.str.strip()

df_inflation = df_inflation.rename(columns={'Inflation Rate (%)': 'inflation_rate'})

# Drop the 'Annual Change' column
df_inflation = df_inflation.drop(columns=['Annual Change', ''])

# Rename the 'date' column to 'year' and extract only the year part
df_inflation['date'] = pd.to_datetime(df_inflation['date'])
df_inflation['year'] = df_inflation['date'].dt.year
df_inflation = df_inflation.drop(columns=['date'])

# Filter the DataFrame for years from 2008 to 2022
df_inflation = df_inflation[df_inflation['year'] >= 2008]

df_inflation.head()


,inflation_rate,year
48,4.0753,2008
49,-0.2878,2009
50,1.7999,2010
51,3.1961,2011
52,2.4461,2012


In [23]:
df_inflation.columns

Index(['inflation_rate', 'year'], dtype='object')

In [142]:
df_employment_rate = pd.read_csv('/content/drive/MyDrive/ADSDB/landing/temporal/employment_rate.csv', sep=';')

df_employment_rate = df_employment_rate.rename(columns={
    'Total': 'labor_rate_value',
    'Periodo': 'Quarter',
    'Rates': 'labor_rate'
})

df_employment_rate['year'] = df_employment_rate['Quarter'].str[:4]

df_employment_rate.head()

,Sex,Provinces,labor_rate,Quarter,labor_rate_value,year
0,Both sexes,National Total,Activity rate,2023QII,58.97,2023
1,Both sexes,National Total,Activity rate,2023QI,58.55,2023
2,Both sexes,National Total,Activity rate,2022QIV,58.52,2022
3,Both sexes,National Total,Activity rate,2022QIII,58.86,2022
4,Both sexes,National Total,Activity rate,2022QII,58.71,2022


In [143]:
df_house_price_index = pd.read_csv('/content/drive/MyDrive/ADSDB/landing/temporal/house_price_index.csv', sep=';')

# Rename the columns
df_house_price_index = df_house_price_index.rename(columns={
    'Total': 'house_price_index',
    'Periodo': 'Quarter'
})

# Adding a new column 'year' which extracts the year from the 'Quarter' column
df_house_price_index['year'] = df_house_price_index['Quarter'].str[:4]


df_house_price_index.head()

,National Total,Autonomous Communities and Cities,Index type,Indices and rates,Quarter,house_price_index,year
0,Nacional,NaN,General,Index,2023QII,146.583,2023
1,Nacional,NaN,General,Index,2023QI,143.580,2023
2,Nacional,NaN,General,Index,2022QIV,142.666,2022
3,Nacional,NaN,General,Index,2022QIII,143.860,2022
4,Nacional,NaN,General,Index,2022QII,141.433,2022


In [39]:
dataframes = [df_inflation, df_employment_rate, df_house_price_index]
for df in dataframes:
  print("--------------------------------")
  print(df.describe)
  for column in df.columns:
      # Check if the column is of categorical type
        print(f'Modalities for {column}:')
        print(df[column].value_counts())
        print("..................................................")  # Just for better readability


--------------------------------
<bound method NDFrame.describe of     inflation_rate  year
48          4.0753  2008
49         -0.2878  2009
50          1.7999  2010
51          3.1961  2011
52          2.4461  2012
53          1.4086  2013
54         -0.1511  2014
55         -0.5004  2015
56         -0.2026  2016
57          1.9561  2017
58          1.6750  2018
59          0.6995  2019
60         -0.3228  2020
61          3.0931  2021
62          8.3906  2022>
Modalities for inflation_rate:
 4.0753    1
-0.2878    1
 1.7999    1
 3.1961    1
 2.4461    1
 1.4086    1
-0.1511    1
-0.5004    1
-0.2026    1
 1.9561    1
 1.6750    1
 0.6995    1
-0.3228    1
 3.0931    1
 8.3906    1
Name: inflation_rate, dtype: int64
..................................................
Modalities for year:
2008    1
2009    1
2010    1
2011    1
2012    1
2013    1
2014    1
2015    1
2016    1
2017    1
2018    1
2019    1
2020    1
2021    1
2022    1
Name: year, dtype: int64
........................

In [110]:
for df in dataframes:
  print("df: ", df.shape)

df:  (15, 2)
df:  (41022, 7)
df:  (15840, 7)


In [146]:
df_inflation['year'] = df_inflation['year'].astype(int)
df_employment_rate['year'] = df_employment_rate['year'].astype(int)
df_house_price_index['year'] = df_house_price_index['year'].astype(int)


inflation_rate_map = df_inflation.set_index('year')['inflation_rate']
df_employment_rate['inflation_rate'] = df_employment_rate['year'].map(inflation_rate_map)

# Now df_employment_rate should have an additional column 'inflation_rate' with values from df_inflation
df_employment_rate.head(), df_employment_rate.shape

(          Sex       Provinces     labor_rate   Quarter labor_rate_value  year  \
 2  Both sexes  National Total  Activity rate   2022QIV            58.52  2022   
 3  Both sexes  National Total  Activity rate  2022QIII            58.86  2022   
 4  Both sexes  National Total  Activity rate   2022QII            58.71  2022   
 5  Both sexes  National Total  Activity rate    2022QI             58.5  2022   
 6  Both sexes  National Total  Activity rate   2021QIV            58.65  2021   
 
    inflation_rate  
 2          8.3906  
 3          8.3906  
 4          8.3906  
 5          8.3906  
 6          3.0931  ,
 (9540, 7))

In [145]:

# Filtering df_employment_rate and df_house_price_index to include only data from 2008 to 2022
df_employment_rate = df_employment_rate[(df_employment_rate['year'] >= 2008) & (df_employment_rate['year'] <= 2022)]
df_house_price_index = df_house_price_index[(df_house_price_index['year'] >= 2008) & (df_house_price_index['year'] <= 2022)]

df_employment_rate = df_employment_rate[df_employment_rate['labor_rate'] == 'Activity rate']
df_employment_rate = df_employment_rate[(df_employment_rate['Provinces'] != '18 Ceuta') & (df_employment_rate['Provinces'] != '19 Melilla')]

df_employment_rate['Provinces'] = df_employment_rate['Provinces'].apply(lambda x: x[2:] if x != "National Total" else x)

df_house_price_index = df_house_price_index[df_house_price_index['Indices and rates'] == 'Index']
df_house_price_index = df_house_price_index.rename(columns={'Autonomous Communities and Cities': 'Provinces'})
df_house_price_index = df_house_price_index[(df_house_price_index['Provinces'] != '18 Ceuta') & (df_house_price_index['Provinces'] != '19 Melilla')]
df_house_price_index['Provinces'].fillna('National Total', inplace=True)
df_house_price_index['Provinces'] = df_house_price_index['Provinces'].apply(lambda x: x[2:] if pd.notnull(x) and x != "National Total" else x)


# Displaying the first few rows of the filtered dataframes and their shapes
(df_employment_rate.head(), df_employment_rate.shape, df_house_price_index.head(), df_house_price_index.shape)


(          Sex       Provinces     labor_rate   Quarter labor_rate_value  year  \
 2  Both sexes  National Total  Activity rate   2022QIV            58.52  2022   
 3  Both sexes  National Total  Activity rate  2022QIII            58.86  2022   
 4  Both sexes  National Total  Activity rate   2022QII            58.71  2022   
 5  Both sexes  National Total  Activity rate    2022QI             58.5  2022   
 6  Both sexes  National Total  Activity rate   2021QIV            58.65  2021   
 
    inflation_rate  
 2          8.3906  
 3          8.3906  
 4          8.3906  
 5          8.3906  
 6          3.0931  ,
 (9540, 7),
   National Total       Provinces Index type Indices and rates   Quarter  \
 2       Nacional  National Total    General             Index   2022QIV   
 3       Nacional  National Total    General             Index  2022QIII   
 4       Nacional  National Total    General             Index   2022QII   
 5       Nacional  National Total    General             Index 

In [157]:
df_employment_rate.isnull().sum()

Sex                 0
Provinces           0
labor_rate          0
Quarter             0
labor_rate_value    0
year                0
inflation_rate      0
dtype: int64

In [156]:
df_house_price_index.isnull().sum()

National Total       0
Provinces            0
Index type           0
Indices and rates    0
Quarter              0
house_price_index    0
year                 0
dtype: int64

In [172]:
df_employment_rate['Provinces'].value_counts()


National Total            180
Rioja. La                 180
Lleida                    180
Lugo                      180
Madrid                    180
Málaga                    180
Murcia                    180
Navarra                   180
Ourense                   180
Palencia                  180
Palmas. Las               180
Pontevedra                180
Salamanca                 180
Jaén                      180
Santa Cruz de Tenerife    180
Segovia                   180
Sevilla                   180
Soria                     180
Tarragona                 180
Teruel                    180
Toledo                    180
Valencia/València         180
Valladolid                180
Zamora                    180
León                      180
Huesca                    180
Albacete                  180
Cáceres                   180
Alicante/Alacant          180
Almería                   180
Araba/Álava               180
Asturias                  180
Ávila                     180
Badajoz   

In [171]:
df_house_price_index['Provinces'].value_counts()


National Total                 180
Andalucía                      180
País Vasco                     180
Navarra. Comunidad Foral de    180
Murcia. Región de              180
Madrid. Comunidad de           180
Galicia                        180
Extremadura                    180
Comunitat Valenciana           180
Cataluña                       180
Castilla - La Mancha           180
Castilla y León                180
Cantabria                      180
Canarias                       180
Balears. Illes                 180
Asturias. Principado de        180
Aragón                         180
Rioja. La                      180
Name: Provinces, dtype: int64

In [160]:
# Extracting unique province names from both dataframes
unique_provinces_house_price_index = set(df_house_price_index['Provinces'].unique())
unique_provinces_employment_rate = set(df_employment_rate['Provinces'].unique())

# Checking if all provinces in house price index are present in employment rate
all_provinces_present = unique_provinces_house_price_index.issubset(unique_provinces_employment_rate)

# Listing provinces not present in employment rate (if any)
provinces_not_present = unique_provinces_house_price_index - unique_provinces_employment_rate

all_provinces_present, provinces_not_present



(False,
 {'Andalucía',
  'Aragón',
  'Asturias. Principado de',
  'Canarias',
  'Castilla - La Mancha',
  'Castilla y León',
  'Cataluña',
  'Comunitat Valenciana',
  'Extremadura',
  'Galicia',
  'Madrid. Comunidad de',
  'Murcia. Región de',
  'Navarra. Comunidad Foral de',
  'País Vasco'})

In [169]:
province_to_autonomous_community_mapping = {
    "National Total": "National Total",
    "Rioja. La": "Rioja. La",
    "Lleida": "Cataluña",
    "Lugo": "Galicia",
    "Madrid": "Madrid. Comunidad de",
    "Málaga": "Andalucía",
    "Murcia": "Murcia. Región de",
    "Navarra": "Navarra. Comunidad Foral de",
    "Ourense": "Galicia",
    "Palencia": "Castilla y León",
    "Palmas. Las": "Canarias",
    "Pontevedra": "Galicia",
    "Salamanca": "Castilla y León",
    "Jaén": "Andalucía",
    "Santa Cruz de Tenerife": "Canarias",
    "Segovia": "Castilla y León",
    "Sevilla": "Andalucía",
    "Soria": "Castilla y León",
    "Tarragona": "Cataluña",
    "Teruel": "Aragón",
    "Toledo": "Castilla - La Mancha",
    "Valencia/València": "Comunitat Valenciana",
    "Valladolid": "Castilla y León",
    "Zamora": "Castilla y León",
    "León": "Castilla y León",
    "Huesca": "Aragón",
    "Albacete": "Castilla - La Mancha",
    "Cáceres": "Extremadura",
    "Alicante/Alacant": "Comunitat Valenciana",
    "Almería": "Andalucía",
    "Araba/Álava": "País Vasco",
    "Asturias": "Asturias. Principado de",
    "Ávila": "Castilla y León",
    "Badajoz": "Extremadura",
    "Balears. Illes": "Balears. Illes",
    "Barcelona": "Cataluña",
    "Bizkaia": "País Vasco",
    "Burgos": "Castilla y León",
    "Cádiz": "Andalucía",
    "Huelva": "Andalucía",
    "Cantabria": "Cantabria",
    "Castellón/Castelló": "Comunitat Valenciana",
    "Ciudad Real": "Castilla - La Mancha",
    "Córdoba": "Andalucía",
    "Coruña. A": "Galicia",
    "Cuenca": "Castilla - La Mancha",
    "Gipuzkoa": "País Vasco",
    "Girona": "Cataluña",
    "Granada": "Andalucía",
    "Guadalajara": "Castilla - La Mancha",
    "Zaragoza": "Aragón",
}

# Example usage:
df_employment_rate['Autonomous Community'] = df_employment_rate['Provinces'].map(province_to_autonomous_community_mapping)


Castilla y León                1620
Andalucía                      1440
Castilla - La Mancha            900
Cataluña                        720
Galicia                         720
Comunitat Valenciana            540
País Vasco                      540
Aragón                          540
Canarias                        360
Extremadura                     360
Madrid. Comunidad de            180
Navarra. Comunidad Foral de     180
Murcia. Región de               180
National Total                  180
Cantabria                       180
Balears. Illes                  180
Asturias. Principado de         180
Rioja. La                       180
Name: Autonomous Community, dtype: int64

In [173]:
df_employment_rate = df_employment_rate.drop(columns=['Provinces'])

df_employment_rate = df_employment_rate.rename(columns={'Autonomous Community': 'Provinces'})

df_employment_rate.isnull().sum()

Sex                 0
labor_rate          0
Quarter             0
labor_rate_value    0
year                0
inflation_rate      0
Provinces           0
dtype: int64

In [174]:
# Merge the two dataframes based on year, quarter, and provinces
merged_df = pd.merge(df_employment_rate, df_house_price_index, on=['year', 'Quarter', 'Provinces'], how='inner')

# Drop duplicate rows (if any)
merged_df.drop_duplicates(inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Display the merged dataframe
print(merged_df)


              Sex     labor_rate  Quarter labor_rate_value  year  \
0      Both sexes  Activity rate  2022QIV            58.52  2022   
1      Both sexes  Activity rate  2022QIV            58.52  2022   
2      Both sexes  Activity rate  2022QIV            58.52  2022   
3           Males  Activity rate  2022QIV            63.53  2022   
4           Males  Activity rate  2022QIV            63.53  2022   
...           ...            ...      ...              ...   ...   
27478       Males  Activity rate   2008QI            68.84  2008   
27479       Males  Activity rate   2008QI            68.84  2008   
27480     Females  Activity rate   2008QI             50.8  2008   
27481     Females  Activity rate   2008QI             50.8  2008   
27482     Females  Activity rate   2008QI             50.8  2008   

       inflation_rate       Provinces National Total            Index type  \
0              8.3906  National Total       Nacional               General   
1              8.3906  Nati

In [193]:
merged_df.isnull().sum()

Year                      0
Quarter                   0
Provinces                 0
Sex                       0
activity_rate             0
inflation_rate            0
house_price_index_type    0
house_price_index         0
dtype: int64

In [192]:
# Define the desired column order

# Reorder the columns in the DataFrame
desired_columns = ['Year', 'Quarter', 'Provinces', 'Sex', 'activity_rate', 'inflation_rate', 'house_price_index_type', 'house_price_index']
merged_df = merged_df[desired_columns]


In [194]:
path = '/content/drive/MyDrive/ADSDB/exploitation/data.csv'
merged_df.to_csv(path, index=False)
